# KKS Binary Solidification with FiPy
This notebook contains code for binary solidification using the
Kim-Kim-Suzuki model [1] for interfacial energy. This allows easy specification
of gamma, but requires constant chemical potential through the interface.
The implementation involves iteratively solving for chemical composition in
pure phases such that the chemical potential constraint is satisfied [2].

Questions/comments to trevor.keller@nist.gov (Trevor Keller).

References:
1. Kim, Kim, and Suzuki. "Phase-field model for binary alloys."
    _Physical Review E_ 60:6;7186-7197 (1999). 
2. Provatas and Elder. _Phase-Field Methods in Materials Science and Engineering_,
    Chapter 6, Section 9. Wiley VCH: Weinheim, Germany. 2010.


## Model Description (paraphrased after Provatas & Elder)
We are setting out to simulate solidification of two-component alloy with a lenticular phase diagram, or "binary isomorphous" system, such as Cu-Ni. The free energy curves for pure phases, $f_S$ and $f_L$, are generated from a CALPHAD database, rather than a regular solution model.

In addition to this thermodynamic description, we are adopting the KKS treatment of diffuse interfaces. This simply means that at equilibrium, chemical potential is constant through the interface, and composition varies to make it so. More concretely, composition is defined by the phase fraction $\phi$ and two fictitious concentration fields, $C_S$ and $C_L$, representing composition of the pure phase, as

$$c = h(\phi)C_S + (1-h(\phi))C_L,$$

where the interpolation function $h(\phi)=\phi^3(6\phi^2-15\phi+10)$ takes the values in solid $h(\phi=1)=1$ and liquid $h(\phi=0)=0$. At equilibrium,

$$\mu = \left.\frac{\partial f_S}{\partial c}\right|_{c=C_S} = \left.\frac{\partial f_L}{\partial c}\right|_{c=C_L}.$$

Taken together, and introducing a double-well function $g(\phi)=\phi^2(1-\phi)^2$, the thermodynamic and interfacial treatments provide the bulk free energy,

$$f(\phi,c,T) = \omega g(\phi) + h(\phi)f_S(C_S,T) + (1-h(\phi))f_L(C_L,T).$$

Now, assuming nonconserved (Allen-Cahn) dynamics for $\phi$ and conserved (Cahn-Hilliard) dynamics for $c$, we can write the equations of motion

$$\frac{\partial\phi}{\partial t} = -M_\phi\frac{\delta\mathcal{F}}{\delta\phi} \rightarrow \tau\frac{\partial\phi}{\partial t} = \epsilon_\phi^2\nabla^2\phi -\omega g'(\phi) + h'(\phi)\left[f_L(C_L) - f_S(C_S) - \frac{\partial f_L(C_L)}{\partial c}(C_L - C_S)\right]$$

$$\frac{\partial c}{\partial t} = \nabla\cdot M_c\nabla\frac{\delta\mathcal{F}}{\delta c} = D_L\nabla\cdot Q(\phi)\left[h(\phi)\nabla C_S + (1-h(\phi))\nabla C_L\right]$$

with phase-dependent mobility $Q(\phi)=\frac{1-\phi}{(1+k) - (1-k)\phi}$, partition coefficient $k=\frac{C_S^e}{C_L^e}$, and time constant $\tau = M_\phi^{-1}$.

### The Wrinkle
$C_S$ and $C_L$ are not constants, they are field variables whose values depend on $\phi$ and $c$. Determining their values requires solving for the common tangent, or the coupled roots

$$f_1(C_S,C_L) = h(\phi)C_S + (1-h(\phi))C_L -c = 0$$

$$f_2(C_S,C_L) = \frac{\partial f_S(C_S)}{\partial c} - \frac{\partial f_L(C_L)}{\partial c} = 0.$$

This is straightforward using Newton's Method for two simultaneous nonlinear equations, discussed in _Numerical Recipes_ and Ref. [2] above. The iterative scheme is

$$ \left(\begin{array}{c} C_S^{n+1}\\ C_L^{n+1}\end{array}\right) = \left(\begin{array}{c} C_S^n\\ C_L^n\end{array}\right) + \frac{1}{\mathrm{det}|\mathbf{J}|}
\left[\begin{array}{c} 
    \frac{\partial^2 f_L(C_L^n)}{\partial c^2}f_1(C_S^n,C_L^n) + (1-h(\phi))f_2(C_S^n,C_L^n)\\
    \frac{\partial^2 f_S(C_S^n)}{\partial c^2}f_1(C_S^n,C_L^n) - h(\phi)f_2(C_S^n,C_L^n)
\end{array}\right]$$

with $\mathrm{det}|\mathbf{J}|=h(\phi)\frac{\partial^2 f_L(C_L^n)}{\partial c^2} + (1-h(\phi))\frac{\partial^2 f_S(C_S^n)}{\partial c^2}$. If your initial guesses are close to the root, this should converge within some tolerance using the error

$$ \eta = \sqrt{\left(f_1(C_s^{n+1},C_l^{n+1})\right)^2 + \left(f_2(C_s^{n+1},C_l^{n+1})\right)^2}. $$

Note that this is not a normalized residual. While some entries in the LUT will cross $\eta<10^{-8}$, many stagnate at $\mathcal{O}(10^{-3})$.

Since these two equations have no spatial dependence (no gradients, etc.), and Newton's method is computationally expensive, the standard approach is to construct a lookup table for $C_S$ and $C_L$ covering $\phi=[-\delta,1+\delta]$ and $c=[-\delta,1+\delta]$ with a reasonably high number of points, e.g. $125\times125$, then interpolating from the LUT at runtime. For best results, use the interpolated values as initial guesses for a touch-up iteration or two.

## Required Libraries

In [ ]:
%matplotlib inline
from fipy import CellVariable, DiffusionTerm, Grid2D, MultiViewer, Solver, TransientTerm, Variable, Viewer
import numpy as np
from matplotlib import pyplot as plt

## Thermodynamics
The free energy curves for solid and liquid were computed using PyCalphad, fit with $10^\mathrm{th}$-order polynomials. Planned upgrades will include equivalent OpenCalphad calls (via PyOC) with SymPy to achieve this without human intervention. For now, we'll risk using the hand-coded result.

In [ ]:
# polynomial coefficients from curve fitting to CALPHAD at 1550 K
calCs = (6.19383857e+03,-3.09926825e+04, 6.69261368e+04,-8.16668934e+04, \
         6.19902973e+04,-3.04134700e+04, 9.74968659e+03,-2.04529002e+03, \
         2.95622845e+02,-3.70962613e+01,-6.12900561e+01)

calCl = (6.18692878e+03,-3.09579439e+04, 6.68516329e+04,-8.15779791e+04, \
         6.19257214e+04,-3.03841489e+04, 9.74145735e+03,-2.04379606e+03, \
         2.94796431e+02,-3.39127135e+01,-6.26373908e+01)

# bulk free energy curves
def fs(c_):
    return   calCs[0]*c_**10 + calCs[1]*c_**9 + calCs[2]*c_**8 + calCs[3]*c_**7 \
           + calCs[4]*c_**6  + calCs[5]*c_**5 + calCs[6]*c_**4 + calCs[7]*c_**3 \
           + calCs[8]*c_**2  + calCs[9]*c_    + calCs[10]

def fl(c_):
    return   calCl[0]*c_**10 + calCl[1]*c_**9 + calCl[2]*c_**8 + calCl[3]*c_**7 \
           + calCl[4]*c_**6  + calCl[5]*c_**5 + calCl[6]*c_**4 + calCl[7]*c_**3 \
           + calCl[8]*c_**2  + calCl[9]*c_    + calCl[10]

# first derivatives
def dfs_dc(c_):
    return  10.0*calCs[0]*c_**9 + 9.0*calCs[1]*c_**8 + 8.0*calCs[2]*c_**7 \
           + 7.0*calCs[3]*c_**6 + 6.0*calCs[4]*c_**5 + 5.0*calCs[5]*c_**4 \
           + 4.0*calCs[6]*c_**3 + 3.0*calCs[7]*c_**2 + 2.0*calCs[8]*c_ + calCs[9];

def dfl_dc(c_):
    return  10.0*calCl[0]*c_**9 + 9.0*calCl[1]*c_**8 + 8.0*calCl[2]*c_**7 \
           + 7.0*calCl[3]*c_**6 + 6.0*calCl[4]*c_**5 + 5.0*calCl[5]*c_**4 \
           + 4.0*calCl[6]*c_**3 + 3.0*calCl[7]*c_**2 + 2.0*calCl[8]*c_ + calCl[9];
            
# second derivatives
def d2fs_dc2(c_):
    return   90.0*calCs[0]*c_**8 + 72.0*calCs[1]*c_**7 + 56.0*calCs[2]*c_**6 \
           + 42.0*calCs[3]*c_**5 + 30.0*calCs[4]*c_**4 + 20.0*calCs[5]*c_**3 \
           + 12.0*calCs[6]*c_**2 +  6.0*calCs[7]*c_    +  2.0*calCs[8]

def d2fl_dc2(c_):
    return   90.0*calCl[0]*c_**8 + 72.0*calCl[1]*c_**7 + 56.0*calCl[2]*c_**6 \
           + 42.0*calCl[3]*c_**5 + 30.0*calCl[4]*c_**4 + 20.0*calCl[5]*c_**3 \
           + 12.0*calCl[6]*c_**2 +  6.0*calCl[7]*c_    +  2.0*calCl[8]

# pure equilibrium compositions (from solving dfs/dc = 0 and dfl/dc = 0)
Cse = 0.48300
Cle = 0.33886

In [ ]:
w = np.linspace(-0.1,1.1,125)
plt.plot(w, fs(w), label=('$f_S$'))
plt.plot(w, fl(w), label=('$f_L$'))
plt.xlabel('$x_\mathrm{Ni}$',fontsize=20)
plt.ylabel('$G$',fontsize=20)
plt.legend(loc=9)

## Fictitious composition fields
$C_S$ and $C_L$ are computed using the thermodynamic potentials for pure phases ($f_S$ and $f_L$), interpolation and double-well functions, and a weighting function.

In [97]:
# Interpolation function
def h(phi_):
    return phi_**3 * (6. * phi_**2 - 15. * phi_ + 10.)

def hprime(phi_):
    return 30. * phi_**2 * (1. - 2. * phi_)

# Double well function
def g(phi_):
    return phi_**2 * (1. - phi_)**2

def gprime(phi_):
    return 2. * phi_ * (2. * phi_ - 1.) * (phi_ - 1.)

# Weighting coefficient (W) for Newton's method
def newt_weight(phi_, Cs_, Cl_):
    return h(phi_) * d2fl_dc2(Cl_) + (1. - h(phi_)) * d2fs_dc2(Cs_)

def conc_equality(phi_, c_, Cs_, Cl_):
    # f1 in the itereative scheme: concentration equality
    return h(phi_) * Cs_ + (1. - h(phi_)) * Cl_ - c_

def diff_equality(phi_, c_, Cs_, Cl_):
    # f2 in the iterative scheme: diffusion (or chemical) potential equality
    return dfs_dc(Cs_) - dfl_dc(Cl_)

### Newton's method
We should have everything we need to compute self-consistent values of $C_S$ and $C_L$ given $\phi$ and $c$.

In [ ]:
# There's probably a library to do this, e.g. scipy.optimize or sympy.

## Initial Conditions
For testing, the domain contains two circular solid seeds of different radii. The entire domain starts with the same composition.

In [ ]:
# Spacetime parameters
nx = ny = 64       # domain size
dx = dy = 0.075    # mesh resolution
dt = Variable(0.1) # initial timestep

# Grid and scalar fields
mesh = Grid2D(nx=nx, ny=ny, dx=dx, dy=dy)
x, y = mesh.cellCenters
phase = CellVariable(mesh=mesh, hasOld=True)
conc  = CellVariable(mesh=mesh, hasOld=True)
Cs    = CellVariable(mesh=mesh, hasOld=False)
Cl    = CellVariable(mesh=mesh, hasOld=False)
# Would a vector-valued variable make more sense than four separate scalar fields?
# Default boundary conditions are... ?

In [ ]:
def initialize():
    # Flat field initializations
    conc[:] = (Cse + Cle)/2.
    phase[:] = 0.
    Cs[:] = (Cse + Cle)/2.
    Cl[:] = (Cse + Cle)/2.
    
    # Initial microstructure
    rA = 20. * dx
    rB = 16. * dx
    cA = (     nx * dx / 3.,      ny * dy / 3.)
    cB = (3. * nx * dx / 4., 3. * ny * dy / 4.)
    mask = ((x - cA[0])**2 + (y - cA[1])**2 < rA**2) + \
           ((x - cB[0])**2 + (y - cB[1])**2 < rB**2)
    phase.setValue(1., where=mask)

initialize()

In [ ]:
# Visualize initial conditions
viewer = MultiViewer(viewers = (Viewer(vars=phase, title='$\phi$'), \
                                Viewer(vars=conc,  title='$c$')))
viewer.plot()

# Can these be forced into a row, instead of a column?

## Read or Generate Lookup Table (LUT)
The fictitious single-phase composition fields $(C_S, C_L)$ are expensive to compute at runtime. Therefore, a LUT is generated offline for quick access to close approximations for the $(\phi,c)$ values at each grid point. This should be written as a tab-delimited table (TSV) of coordinates $(\phi,c)$ and values $(C_S,C_L)$.

If any parameters affecting the free energy landscape change, the LUT must be re-generated.

In [ ]:
# Attempt to open consistentC.tsv. If it does not exist, create it. 
try:
    with open('consistentC.tsv') as file:
        # LUT already exists: no need to generate
        pass
except IOError as e:
    print "Lookup table does not exist: generating from scratch. Please be patient..."
    # What's the Pythonic way to do this? NumPy array in 2D? No, need coords.
    # Does NumPy have a handy TSV writer? (yes, savetxt) Does FiPy? (yes, tsvviewer)
    #
    # Pseudocode:
    # create a mesh over phi and c, 125x125 points
    # create a pair of variables (or two-element vector) at each point, representing Cs and Cl
    # for c in -0.1..1.1:
    #    for phi in -0.1..1.1:
    #        initialize Cs and Cl with reasonable guesses: average of equilibrium compositions?
    #        while residual is above tolerance and number of iterations is below patience limit:
    #            apply Newton's method holding c, phi constant to determine Cs, Cl, residual
    #            if iterations wander off to infinity:
    #                randomize Cs, Cl
    # export Cs, Cl to TSV

# LUT either already existed, or has now been written to disk. Read it into memory.
# Note that even in distributed parallel (MPI), each rank must have the full LUT in memory.




    